In [1]:
import os
os.chdir("/home/gridsan/qwang/urban-control/")

import pandas as pd
from glob import glob

In [2]:
validation_set_path = [
    'data/train/20250416_v1_validation.csv', 
    'data/train/20250416_v2_validation.csv', 
    'data/train/20250416_v3_validation.csv', 
    'data/train/20250416_v4_validation.csv', 
]

In [25]:
df_arr = []
for i in validation_set_path:
    df_tmp = pd.read_csv(i)
    df_tmp = df_tmp[['city_name', 'x', 'y', 'x_offset', 'y_offset', 'image_column', 'conditioning_image_column']]
    df_arr.append(df_tmp)

df = pd.concat(df_arr).drop_duplicates()

In [26]:
df.to_csv('data/validation/20250416_validation_set.csv', index=False)

In [27]:
for i in range(1,5):
    df_train_tmp = pd.read_csv(f'data/train/20250416_v{i}_train.csv')
    df_validation_tmp = pd.read_csv(f'data/train/20250416_v{i}_validation.csv')
    df_tmp = pd.concat([df_train_tmp, df_validation_tmp])
    if i == 4:
        df_tmp = df_tmp.rename(columns = {'llm_caption': f'caption_v{i}'})[['city_name', 'x', 'y', 'x_offset', 'y_offset', 'image_column', 'conditioning_image_column', f'caption_v{i}']]
    else:
        df_tmp = df_tmp.rename(columns = {'caption': f'caption_v{i}'})[['city_name', 'x', 'y', 'x_offset', 'y_offset', 'image_column', 'conditioning_image_column', f'caption_v{i}']]
    df = pd.merge(df, df_tmp, on = ['city_name', 'x', 'y', 'x_offset', 'y_offset', 'image_column', 'conditioning_image_column'], how='left')

In [28]:
df['caption_v3'] = df['caption_v3'].str.replace('City:', 'Satellite image in a city in')

In [29]:
df.columns

Index(['city_name', 'x', 'y', 'x_offset', 'y_offset', 'image_column',
       'conditioning_image_column', 'caption_v1', 'caption_v2', 'caption_v3',
       'caption_v4'],
      dtype='object')

In [30]:
for i in range(1,5):
    df_export = df.dropna(subset=["caption_v"+str(i)])
    df_export = df_export.rename(columns={"caption_v"+str(i):"caption"})
    df_export = df_export[['city_name', 'x', 'y', 'x_offset', 'y_offset', 'image_column', 'conditioning_image_column', 'caption']]
    
    df_export.to_csv(f'data/validation/20250416_validation_v{str(i)}.csv', index=False)
    